In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import (
    ttest_ind, mannwhitneyu, chi2_contingency, fisher_exact,
    shapiro, levene
)
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# === FILE PATHS ===
project_path = '/Users/labneuro2/Documents/lab/AI_MBS/AI_MBS'

# === LOAD DATA ===
df_features_reg = pd.read_excel(f'{project_path}/local_measures_combined_atlas.xlsx') \
    .sort_values(by='Subject').reset_index(drop=True)

df_clinic = pd.read_excel(f'{project_path}/clinical_data.xlsx') \
    .sort_values(by='subject')

df_features_corr = pd.read_csv(f'{project_path}/FC_combined_atlas.csv') \
    .sort_values(by='Subject').reset_index(drop=True)

# === LABELS: weight loss success (>50% overweight lost within a year) ===
y = (df_clinic.loc[df_clinic['post_MBS'] == 1]['overweight_delta_proc_to_baseline_kg'] < -50).astype(int).to_numpy()
df_clinic = df_clinic[df_clinic['post_MBS'] == 0].reset_index(drop=True)

In [ ]:
# === FUNCTION: Chi² or Fisher's test for categorical variables ===
def compute_chi2_fdr_or_fisher(df, y, cat_vars):
    results = []
    for var in cat_vars:
        contingency = pd.crosstab(df[var], y)
        try:
            chi2, p, _, expected = chi2_contingency(contingency)
            if (expected < 1).any() and contingency.shape == (2, 2):
                stat, p = fisher_exact(contingency)
                test, note = 'Fisher', 'expected < 1'
            elif (expected < 5).sum() / expected.size > 0.2 and contingency.shape == (2, 2):
                stat, p = fisher_exact(contingency)
                test, note = 'Fisher', 'many cells < 5'
            else:
                stat, test, note = chi2, 'Chi2', ''
        except Exception as e:
            stat, p, test, note = np.nan, np.nan, 'Error', str(e)
        results.append([var, stat, p, test, note])
    df_result = pd.DataFrame(results, columns=['Feature', 'Statistic', 'p_value', 'Test', 'Note'])
    df_result['p_value_fdr'] = multipletests(df_result['p_value'], method='fdr_bh')[1]
    return df_result.sort_values(by='p_value').reset_index(drop=True)

# === FUNCTION: t-test or Mann-Whitney test for continuous variables ===
def compute_t_or_nonparametric_fdr(X, y, feature_names, alpha=0.05):
    results = []
    for i, name in enumerate(feature_names):
        group0, group1 = X[y == 0, i], X[y == 1, i]
        p_sh0 = shapiro(group0).pvalue if len(group0) >= 3 else 1.0
        p_sh1 = shapiro(group1).pvalue if len(group1) >= 3 else 1.0
        normal = p_sh0 > alpha and p_sh1 > alpha
        equal_var = levene(group0, group1).pvalue > alpha
        if normal:
            stat, p_val = ttest_ind(group0, group1, equal_var=equal_var, nan_policy='omit')
            test, note = 't-test', 'normal'
        else:
            stat, p_val = mannwhitneyu(group0, group1)
            test, note = 'Mann-Whitney', 'non-normal'
        results.append([name, stat, p_val, test, note])
    df = pd.DataFrame(results, columns=['Feature', 'Statistic', 'p_value', 'Test', 'Note'])
    df['p_value_fdr'] = multipletests(df['p_value'], method='fdr_bh')[1]
    return df.sort_values(by='p_value').reset_index(drop=True)

In [ ]:
# === ANALYSIS OF CLINICAL VARIABLES ===
categorical_vars = ['type_2_diabetes', 'depression', 'hypothyroidism', 'female_gender']
results_chi2 = compute_chi2_fdr_or_fisher(df_clinic, y, categorical_vars)

clinic_cont_cols = ['age_y', 'BMI_kgm2', 'waist_cm', 'hips_cm', 'waist_hip_ratio']
X_clinic_cont = df_clinic[clinic_cont_cols].to_numpy()
results_clinic_cont = compute_t_or_nonparametric_fdr(X_clinic_cont, y, clinic_cont_cols)

results_clinic_cont

,Feature,Statistic,p_value,Test,Note,p_value_fdr
0,waist_cm,2.165283,0.035959,t-test,normal,0.060479
1,hips_cm,2.194627,0.036252,t-test,normal,0.060479
2,BMI_kgm2,2.189687,0.036287,t-test,normal,0.060479
3,age_y,1.872549,0.067940,t-test,normal,0.084926
4,waist_hip_ratio,0.247406,0.805772,t-test,normal,0.805772


In [ ]:

# === GROUPS OF fMRI FEATURES ===
groups = {'fALFF': [], 'ALFF': [], 'ReHo': []}
for col in df_features_reg.columns:
    prefix = col.split('_')[0]
    if prefix in groups:
        groups[prefix].append(col)

# Extract feature matrices for each modality
X_falff = df_features_reg[groups['fALFF']].to_numpy()
X_alff = df_features_reg[groups['ALFF']].to_numpy()
X_reho = df_features_reg[groups['ReHo']].to_numpy()
X_corr = df_features_corr[[c for c in df_features_corr.columns if '_to_' in c]].to_numpy()

In [ ]:
# === FUNCTION: parametric and nonparametric comparison for fMRI ===
def compare_param_vs_nonparam(X, y, feature_names):
    results = []
    for i, name in enumerate(feature_names):
        t_stat, t_p = ttest_ind(X[y == 0, i], X[y == 1, i], nan_policy='omit')
        u_stat, u_p = mannwhitneyu(X[y == 0, i], X[y == 1, i])
        results.append([name, t_stat, t_p, u_stat, u_p])
    df = pd.DataFrame(results, columns=['Feature', 't_stat', 't_p', 'u_stat', 'u_p'])
    df['t_p_fdr'] = multipletests(df['t_p'], method='fdr_bh')[1]
    df['u_p_fdr'] = multipletests(df['u_p'], method='fdr_bh')[1]
    return df.sort_values(by='t_p').reset_index(drop=True)

In [ ]:
# === fMRI ANALYSIS ===
results_falff = compare_param_vs_nonparam(X_falff, y, groups['fALFF'])
results_alff = compare_param_vs_nonparam(X_alff, y, groups['ALFF'])
results_reho = compare_param_vs_nonparam(X_reho, y, groups['ReHo'])
results_corr = compare_param_vs_nonparam(X_corr, y, df_features_corr.columns[1:])

In [ ]:
# === FUNCTION: PCA and correlations ===
def apply_pca(X, feature_names):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    pca = PCA()
    X_pca = pca.fit_transform(X_scaled)
    new_names = [f'PC{i+1}' for i in range(X_pca.shape[1])]
    return X_pca, new_names, pca, feature_names

def most_correlated_features(pca, feature_names, component_idx, top_n=20):
    # Get absolute loadings for the selected component
    loadings = np.abs(pca.components_[component_idx])
    sorted_idx = np.argsort(loadings)[::-1][:top_n]
    # Return the top_n most correlated feature names
    return [feature_names[i] for i in sorted_idx]

def compare_param_vs_nonparam(X, y, feature_names):
    # Compare groups using t-test and Mann-Whitney for each feature
    results = []
    for i, name in enumerate(feature_names):
        t_stat, t_p = ttest_ind(X[y == 0, i], X[y == 1, i], nan_policy='omit')
        u_stat, u_p = mannwhitneyu(X[y == 0, i], X[y == 1, i])
        results.append([name, t_stat, t_p, u_stat, u_p])
    df = pd.DataFrame(results, columns=['Feature', 't_stat', 't_p', 'u_stat', 'u_p'])
    # FDR correction for multiple testing
    df['t_p_fdr'] = multipletests(df['t_p'], method='fdr_bh')[1]
    df['u_p_fdr'] = multipletests(df['u_p'], method='fdr_bh')[1]
    return df.sort_values(by='t_p').reset_index(drop=True)

In [ ]:
# === MODALITIES DICTIONARY ===
modalities = {
    'fALFF': (X_falff, groups['fALFF']),
    'ALFF': (X_alff, groups['ALFF']),
    'ReHo': (X_reho, groups['ReHo']),
    'Clinical': (X_clinic_cont, clinic_cont_cols),
    'Connectivity': (X_corr, df_features_corr.columns[1:])
}

# === LOOP: PCA + TESTS + DISPLAY RESULTS ===
pca_results = {}

for modality, (X, feature_names) in modalities.items():
    X_pca, pca_names, pca_model, orig_names = apply_pca(X, feature_names)
    result_df = compare_param_vs_nonparam(X_pca, y, pca_names)
    pca_results[modality] = result_df


In [14]:
# === RAW FEATURE UNIVARIATE TESTS FOR EACH MODALITY ===
modalities_raw = {
    'fALFF': (X_falff, groups['fALFF']),
    'ALFF': (X_alff, groups['ALFF']),
    'ReHo': (X_reho, groups['ReHo']),
    'Connectivity': (X_corr, df_features_corr.columns[1:])
}

# Loop through each modality and run univariate tests on raw features
results_raw = {}

for modality, (X, feature_names) in modalities_raw.items():
    print(f"\n{'='*30}\n🔬 Raw features - {modality}\n{'='*30}")
    res = compare_param_vs_nonparam(X, y, feature_names)
    print(res[['Feature', 't_stat', 't_p', 't_p_fdr', 'u_stat', 'u_p', 'u_p_fdr']].head(10))
    results_raw[modality] = res


🔬 Raw features - fALFF
                                  Feature    t_stat       t_p   t_p_fdr  \
0           fALFF_17Networks_LH_SomMotA_2  2.679694  0.010402  0.951773   
1           fALFF_17Networks_RH_TempPar_1  2.454769  0.018220  0.951773   
2      fALFF_17Networks_LH_DefaultC_Rsp_3 -2.438614  0.018948  0.951773   
3  fALFF_17Networks_RH_SalVentAttnA_PrC_1 -2.417347  0.019947  0.951773   
4      fALFF_17Networks_LH_SomMotB_Cent_5  2.375902  0.022032  0.951773   
5                            fALFF_PUT-lh -2.285642  0.027265  0.951773   
6           fALFF_17Networks_LH_TempPar_5  2.236420  0.030564  0.951773   
7           fALFF_17Networks_RH_TempPar_2  2.223069  0.031518  0.951773   
8     fALFF_17Networks_RH_DefaultA_PFCd_1 -2.127145  0.039184  0.951773   
9   fALFF_17Networks_LH_DorsAttnB_PostC_4 -2.026234  0.048973  0.951773   

   u_stat       u_p  u_p_fdr  
0   363.0  0.010179  0.93205  
1   349.0  0.024455  0.93205  
2   159.0  0.038719  0.93205  
3   149.0  0.021700  0.932

In [16]:
print("\n" + "="*40)
print("Clinical results - categorical variables")
print("="*40)
print(results_chi2[['Feature', 'Statistic', 'p_value', 'p_value_fdr', 'Test', 'Note']].head(10))

print("\n" + "="*40)
print("Clinical results - continuous variables")
print("="*40)
print(results_clinic_cont[['Feature', 'Statistic', 'p_value', 'p_value_fdr', 'Test', 'Note']].head(10))



Clinical results - categorical variables
           Feature  Statistic   p_value  p_value_fdr    Test            Note
0  type_2_diabetes   2.160511  0.141598      0.56639    Chi2                
1       depression   0.772727  1.000000      1.00000  Fisher  many cells < 5
2   hypothyroidism   0.000000  1.000000      1.00000    Chi2                
3    female_gender   1.055556  1.000000      1.00000  Fisher  many cells < 5

Clinical results - continuous variables
           Feature  Statistic   p_value  p_value_fdr    Test    Note
0         waist_cm   2.165283  0.035959     0.060479  t-test  normal
1          hips_cm   2.194627  0.036252     0.060479  t-test  normal
2         BMI_kgm2   2.189687  0.036287     0.060479  t-test  normal
3            age_y   1.872549  0.067940     0.084926  t-test  normal
4  waist_hip_ratio   0.247406  0.805772     0.805772  t-test  normal


In [17]:
# Mapping of original column names to more descriptive names for Excel export
column_rename_map = {
    'Feature': 'Variable',
    'Statistic': 'Test Statistic',
    't_stat': 't Statistic',
    't_p': 't-test p-value',
    't_p_fdr': 't-test p-value (FDR)',
    'u_stat': 'U Statistic',
    'u_p': 'Mann–Whitney p-value',
    'u_p_fdr': 'Mann–Whitney p-value (FDR)',
    'p_value': 'Raw p-value',
    'p_value_fdr': 'p-value (FDR-corrected)',
    'Test': 'Test Type',
    'Note': 'Note / Assumption Violation'
}

with pd.ExcelWriter(f"{project_path}/top_features_univariate_results.xlsx", engine='openpyxl') as writer:
    # --- Clinical categorical variables ---
    results_chi2.rename(columns=column_rename_map).to_excel(
        writer, sheet_name="Clinical_Categorical", index=False)

    # --- Clinical continuous variables ---
    results_clinic_cont.rename(columns=column_rename_map).to_excel(
        writer, sheet_name="Clinical_Continuous", index=False)

    # --- Raw features for each modality ---
    for modality, df in results_raw.items():
        df_out = df[['Feature', 't_stat', 't_p', 't_p_fdr', 'u_stat', 'u_p', 'u_p_fdr']]
        df_out.rename(columns=column_rename_map).to_excel(
            writer, sheet_name=f"{modality}_Raw", index=False)

    # --- PCA results for each modality ---
    for modality, df in pca_results.items():
        df_out = df[['Feature', 't_stat', 't_p', 't_p_fdr', 'u_stat', 'u_p', 'u_p_fdr']]
        df_out.rename(columns=column_rename_map).to_excel(
            writer, sheet_name=f"{modality}_PCA", index=False)
